In [1182]:
from github import Github,Auth
from IPython.display import HTML
from decouple import config
from jinja2 import Environment, FileSystemLoader


In [1183]:
g = Github(auth=Auth.Token(config("GITHUB_TOKEN")))

In [1184]:
g.get_user().name

'Giorgio Caizzi'

In [1185]:
g.get_user().get_repos().totalCount

18

In [1186]:
repos = [repo for repo in g.get_user().get_repos() if repo.name in ["pysurfline","py3dinterpolations","repomanager"]]

In [1187]:
len(repos)

3

In [1188]:
repos[0].get_issues().totalCount

9

In [1189]:
repos[0].get_workflows()[0].__dict__

{'_requester': <github.Requester.Requester at 0x10d1d0b90>,
 '_id': <github.GithubObject._ValuedAttribute at 0x10cf89810>,
 '_name': <github.GithubObject._ValuedAttribute at 0x10cb48d10>,
 '_path': <github.GithubObject._ValuedAttribute at 0x10d197690>,
 '_state': <github.GithubObject._ValuedAttribute at 0x10d197b10>,
 '_created_at': <github.GithubObject._ValuedAttribute at 0x10d194a50>,
 '_updated_at': <github.GithubObject._ValuedAttribute at 0x10d196710>,
 '_url': <github.GithubObject._ValuedAttribute at 0x10d196210>,
 '_html_url': <github.GithubObject._ValuedAttribute at 0x10d196610>,
 '_badge_url': <github.GithubObject._ValuedAttribute at 0x10d196590>,
 '_headers': {'server': 'GitHub.com',
  'date': 'Thu, 09 Nov 2023 00:37:34 GMT',
  'content-type': 'application/json; charset=utf-8',
  'transfer-encoding': 'chunked',
  'cache-control': 'private, max-age=60, s-maxage=60',
  'vary': 'Accept, Authorization, Cookie, X-GitHub-OTP, Accept-Encoding, Accept, X-Requested-With',
  'etag': 'W/

In [1190]:
repo_contents = [
    {
        "Name": repo.name,
        "Language": repo.language,
        "Description": repo.description,
        "Url": repo.html_url,
        "Stars": repo.stargazers_count,
        "Issues": {
            "Open": sum([issue.state == "open" for issue in repo.get_issues()]),
            "Total": repo.get_issues().totalCount,
        },
        "Pull Requests": {
            "Open": sum([issue.state == "open" for issue in repo.get_pulls()]),
            "Total": repo.get_pulls().totalCount,
        },
        "Branches": {
            "Default": repo.default_branch,
            "Branches": [branch.name for branch in repo.get_branches()],
            "Total": repo.get_branches().totalCount,
        },
        "Forks": {
            "Forks": [fork.full_name for fork in repo.get_forks()],
            "Total": repo.forks_count,
        },
        "Pages": repo.has_pages,
        "Collaborators": {
            "Collaborators": [
                collaborator.login for collaborator in repo.get_collaborators()
            ],
            "Total": repo.get_collaborators().totalCount,
        },
        "Workflows": {
            "Workflows": [workflow.name for workflow in repo.get_workflows()],
            "Total": repo.get_workflows().totalCount,
        },
    }
    for repo in repos
]

In [1191]:
# Set up the environment
env = Environment(loader=FileSystemLoader('.'),extensions=['jinja2.ext.loopcontrols'])
template = env.get_template('template.html')

In [1192]:
# custom filter jinja2

def is_true(value):
    return isinstance(value, bool) and value

def is_false(value):
    return isinstance(value, bool) and not value

# Add the filter to the Jinja2 environment
env.filters['is_true'] = is_true
env.filters['is_false'] = is_false

In [1193]:
# html = template.render(repos=[vars(repo) for repo in repos])
html = template.render(repos=repo_contents)

In [1194]:
HTML(html)

In [1195]:
with open('index.html', 'w') as file:
    file.write(html)